# gpu/ram/mount

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Nov 30 03:19:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    13W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')


Mounted at /content/drive


# impot

In [78]:
import random
import pandas as pd
import numpy as np
import os
import glob
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import xgboost
from xgboost import XGBRegressor

# make dataset

In [43]:
def preprocessing(input_path):
  all_input_list = sorted(glob.glob(input_path))
  train = pd.DataFrame()
  for datapath in all_input_list:
    data = pd.read_csv(datapath) # 
    data['obs_time'] = data.index % 24 # 시간통일
    df = abs(data)
    df.loc[(df['내부온도관측치'] > 40), '내부온도관측치'] = 40
    df.loc[(df['내부습도관측치'] > 100), '내부습도관측치'] = 100
    df.loc[(df['co2관측치'] > 1200), 'co2관측치'] = 1200
    df.loc[(df['ec관측치'] > 8), 'ec관측치'] = 8
    df.loc[(df['시간당분무량'] > 3000), '시간당분무량'] = 3000
    df.loc[(df['시간당백색광량'] > 120000), '시간당백색광량'] = 120000
    df.loc[(df['시간당적색광량'] > 120000), '시간당적색광량'] = 120000
    df.loc[(df['시간당청색광량'] > 120000), '시간당청색광량'] = 120000
    df.loc[(df['시간당총광량'] > 120000), '시간당총광량'] = 120000
    col_list = df.columns
    for i in range(0,len(col_list)):
      col = col_list[i]    
      if '누적' in col :
        df[col] = df.groupby((df.obs_time == 0).cumsum()).agg(col_list[i-1]).cumsum()
    df.to_csv(datapath,index=False)
    train = pd.concat([train,df])
  print('finish!!')
  return train

In [ ]:
all_input_list = sorted(glob.glob('drive/MyDrive/dacon/data/train_input/*.csv'))
all_input_list

In [49]:
traininput = preprocessing('drive/MyDrive/dacon/data/train_input/*.csv')

finish!!


In [50]:
testinput = preprocessing('drive/MyDrive/dacon/data/test_input/*.csv')

finish!!


In [16]:
colname = traininput.columns

In [93]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
trainsd = scaler.fit(traininput.iloc[:,2:]).transform(traininput.iloc[:,2:])
testsd = scaler.fit(traininput.iloc[:,2:]).transform(testinput.iloc[:,2:])

trainsd = pd.DataFrame(trainsd,columns=colname[2:]).reset_index(drop=True)
testsd = pd.DataFrame(testsd,columns=colname[2:]).reset_index(drop=True)

train=pd.merge(traininput.iloc[:,:2].reset_index(drop=True),trainsd,left_index=True, right_index=True)
test=pd.merge(testinput.iloc[:,:2].reset_index(drop=True),testsd,left_index=True, right_index=True)

In [97]:
test

,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량
0,0,0,0.058938,-0.328692,-0.228843,-0.459312,-0.962502,-0.922162,-0.775478,-0.847469,-0.535571,-0.529592,-0.502909,-0.510168,-1.002462,-1.080641
1,0,1,0.122742,-0.343517,-0.223428,-0.457781,0.450708,-0.810816,-0.775478,-0.847469,-0.535571,-0.529592,-0.502909,-0.510168,-1.002462,-1.080641
2,0,2,-0.034793,-0.359430,-0.220355,-0.454223,-0.962502,-0.810816,-0.775478,-0.847469,-0.535571,-0.529592,-0.502909,-0.510168,-1.002462,-1.080641
3,0,3,-0.039437,-0.335554,-0.186843,-0.453914,0.450708,-0.699470,-0.775478,-0.847469,-0.535571,-0.529592,-0.502909,-0.510168,-1.002462,-1.080641
4,0,4,0.083426,-0.324689,-0.255623,-0.453126,-0.962502,-0.699470,-0.775478,-0.847469,-0.535571,-0.529592,-0.502909,-0.510168,-1.002462,-1.080641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3355,27,19,-0.392404,-0.406467,-0.487867,-0.567553,-0.962502,-0.546486,-0.774720,0.854957,-0.535571,0.117273,-0.502909,-0.133299,-1.001784,0.759057
3356,27,20,-0.352294,-0.480810,-0.511135,-0.566843,-0.813665,-0.534759,-0.775478,0.854957,-0.535571,0.117273,-0.502909,-0.133299,-1.002462,0.759057
3357,27,21,-0.487401,-0.528990,-0.556940,-0.566985,-0.962502,-0.534759,-0.775478,0.854957,-0.535571,0.117273,-0.502909,-0.133299,-1.002462,0.759057
3358,27,22,-0.541866,-0.573025,-0.617379,-0.565806,-0.730949,-0.516515,-0.775478,0.854957,-0.535571,0.117273,-0.502909,-0.133299,-1.002462,0.759057


In [98]:
def preprocessing2(input_path, data):
  all_input_list = sorted(glob.glob(input_path))
  for i in range(0,len(all_input_list)):
    df = data.iloc[ 24*28*i : 24*28*(i+1)]
    df.to_csv(all_input_list[i],index=False)

In [86]:
preprocessing2('drive/MyDrive/dacon/data/train_input/*.csv',train)

In [101]:
preprocessing2('drive/MyDrive/dacon/data/test_input/*.csv',test)

In [119]:
def make_dataset(input_path, target_path):
    train = pd.DataFrame()
    all_input_list = sorted(glob.glob(input_path))
    all_target_list = sorted(glob.glob(target_path))
    for x, y in zip(all_input_list,all_target_list):
        x = pd.read_csv(x)
        y = pd.read_csv(y)
        x = x.iloc[:,1:]
        col_list = x.columns[1:]
        for i in range(0,28) :
            day = x.iloc[24*i:24*i+24]

            day_white = day[day['일간누적백색광량']>=0]['일간누적백색광량'].max()
            day_water = day[day['일간누적분무량']>=0]['일간누적분무량'].max()
            day_light = day[day['일간누적총광량']>=0]['일간누적총광량'].max()
            day_blue = day[day['일간누적청색광량']>=0]['일간누적청색광량'].max()
            day_red = day[day['일간누적적색광량']>=0]['일간누적적색광량'].max()

            day_white2 = day[day['일간누적백색광량']>=0]['일간누적백색광량'].mean()
            day_water2 = day[day['일간누적분무량']>=0]['일간누적분무량'].mean()
            day_light2 = day[day['일간누적총광량']>=0]['일간누적총광량'].mean()
            day_blue2 = day[day['일간누적청색광량']>=0]['일간누적청색광량'].mean()
            day_red2 = day[day['일간누적적색광량']>=0]['일간누적적색광량'].mean()

            time_list = day['obs_time'].unique()
            for col in col_list :
              if col in ['일간누적백색광량','일간누적총광량','일간누적청색광량','일간누적적색광량','일간누적분무량']:
                continue
              else :
                  for t in range(0,len(time_list)-2) :
                      time = time_list[t]
                      ntime = time_list[t+1]
                      nntime = time_list[t+1]
                      value1 = day[day['obs_time']==time][col].iloc[0]
                      value2 = day[day['obs_time']==ntime][col].iloc[0]
                      value3 = day[day['obs_time']==nntime][col].iloc[0]
                      x[col+str(time)+str("+")+str(ntime)+str("+")+str(ntime)] = value1 + value2 + value3

            x['day_water_weight_max'] = (day_water*(i+1))
            x['day_white_weight_max'] = (day_white*(i+1))
            x['day_red_weight_max'] = (day_red*(i+1))
            x['day_blue_weight_max'] = (day_blue*(i+1))
            x['day_light_weight_max'] = (day_light*(i+1))

            x['day_water_weight_mean'] = (day_water2*(i+1))
            x['day_white_weight_mean'] = (day_white2*(i+1))
            x['day_red_weight_mean'] = (day_red2*(i+1))
            x['day_blue_weight_mean'] = (day_blue2*(i+1))
            x['day_light_weight_mean'] = (day_light2*(i+1))

            nx = x.iloc[:1,15:]
            ny = y.iloc[i:i+1].reset_index(drop=True)
            xy = pd.merge(nx,ny,left_index=True, right_index=True)
            train = pd.concat([train,xy]).reset_index(drop=True)

    return train

In [116]:
colname

Index(['DAT', 'obs_time', '내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량',
       '일간누적분무량', '시간당백색광량', '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량',
       '일간누적청색광량', '시간당총광량', '일간누적총광량'],
      dtype='object')

In [120]:
train_input_path, train_target_path = 'drive/MyDrive/dacon/data/train_input/*.csv', 'drive/MyDrive/dacon/data/train_target/*.csv'
test_input_path, test_target_path = 'drive/MyDrive/dacon/data/test_input/*.csv', 'drive/MyDrive/dacon/data/test_target/*.csv'

train = make_dataset(train_input_path, train_target_path)
test = make_dataset(test_input_path, test_target_path)

In [121]:
train=train.fillna(0)
test=test.fillna(0)

In [122]:
train.to_csv('drive/MyDrive/dacon/data/train_standardscale_weight_max_mean_time3.csv',index=False)
test.to_csv('drive/MyDrive/dacon/data/test_standardscale_weight_max_mean_time3.csv',index=False)

In [111]:
train

,내부온도관측치0+1,내부온도관측치1+2,내부온도관측치2+3,내부온도관측치3+4,내부온도관측치4+5,내부온도관측치5+6,내부온도관측치6+7,내부온도관측치7+8,내부온도관측치8+9,내부온도관측치9+10,...,day_red_weight_max,day_blue_weight_max,day_light_weight_max,day_water_weight_mean,day_white_weight_mean,day_red_weight_mean,day_blue_weight_mean,day_light_weight_mean,DAT,predicted_weight_g
0,-0.272155,-0.278910,-0.361332,-0.331355,-0.326288,-0.061563,0.339958,0.515176,0.618195,0.741058,...,0.000000,0.0,1.681538,0.000000,1.432481,0.000000,0.0,1.129978,1,0.167719
1,-0.067634,-0.124210,-0.251979,-0.290401,-0.308134,0.115765,0.713192,0.946252,1.055604,1.137090,...,0.000000,0.0,3.376542,0.000000,2.874379,0.000000,0.0,2.268493,2,0.181787
2,0.090490,0.034758,-0.096607,-0.151916,-0.200892,0.236939,0.861810,1.074603,1.012539,0.709392,...,0.000000,0.0,5.045330,0.000000,4.293981,0.000000,0.0,3.386230,3,0.265921
3,-0.211097,-0.240652,-0.385398,-0.397220,-0.388775,0.021190,0.584840,0.521086,0.244875,0.187455,...,0.000000,0.0,6.753084,0.000000,5.748758,0.000000,0.0,4.536985,4,0.423650
4,-0.284225,-0.301536,-0.334733,-0.319533,-0.299689,-0.064096,0.262272,0.367402,0.428200,0.505465,...,0.000000,0.0,8.414616,0.000000,7.162739,0.000000,0.0,5.649450,5,0.475272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,2.197946,2.299276,2.370787,2.415964,2.461984,2.509694,2.686600,3.028590,2.681533,1.316952,...,0.000000,0.0,0.000000,49.822778,2.934070,0.000000,0.0,0.000000,24,64.875499
780,-1.179851,-1.144808,-1.159308,-1.199840,-1.224751,-1.206596,-0.921182,-0.401020,-0.132916,0.125476,...,3.002824,0.0,18.990891,0.000000,15.749966,2.646990,0.0,14.401342,25,74.002614
781,-1.000761,-0.955163,-0.948626,-0.998024,-1.049112,-1.068111,-0.859117,-0.350354,-0.028208,-0.051430,...,3.107921,0.0,19.740670,0.000000,16.352349,2.733787,0.0,14.943655,26,76.342275
782,-1.018290,-1.016179,-1.017446,-1.040667,-1.055867,-1.060089,-0.879805,-0.461818,-0.230447,-0.190759,...,3.185875,0.0,20.483477,0.000000,16.992838,2.802550,0.0,15.512244,27,82.621245


In [135]:
train=pd.read_csv('drive/MyDrive/dacon/data/train_standardscale_weight_max_mean.csv')
test=pd.read_csv('drive/MyDrive/dacon/data/test_standardscale_weight_max_mean.csv')

In [115]:
import catboost
X = train.drop(['predicted_weight_g'],axis=1)
y = train['predicted_weight_g']
model= {}
for f in range(0,27):
      train_idx = [i for i in range(0,784)]
      valid_idx = train_idx[f*28:f*28+28]
      for idx in valid_idx:
        train_idx.remove(idx)
      print(f'===================================={f+1}============================================')
      print(min(valid_idx),max(valid_idx))
      x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[valid_idx], y.iloc[train_idx], y.iloc[valid_idx]
      
      xgb_model = catboost.CatBoostRegressor(task_type='GPU',verbose=0,random_state=404)
      xgb_model.fit(x_train, y_train)
      
      y_pred = xgb_model.predict(x_val)
      mae = mean_squared_error(y_val, y_pred)**0.5
      print(f"{f + 1} Fold MAE = {mae}")
      model[f] = xgb_model
      print(f'================================================================================\n\n')

====================================1============================================
0 27
1 Fold MAE = 14.15910110945447


====================================2============================================
28 55
2 Fold MAE = 6.545429457161425


====================================3============================================
56 83
3 Fold MAE = 27.653357786108828


====================================4============================================
84 111
4 Fold MAE = 25.029537489830034


====================================5============================================
112 139
5 Fold MAE = 28.382627625806933


====================================6============================================
140 167
6 Fold MAE = 11.533225753110338


====================================7============================================
168 195
7 Fold MAE = 8.94974002807985


====================================8============================================
196 223
8 Fold MAE = 8.937605561428096


=====================

In [117]:
testX = test.drop(['predicted_weight_g'],axis=1)
pc=[0]*140
for fold in range(27):
    pc += model[fold].predict(testX)/27
pc

array([  3.07739323,   3.36578701,   3.34378055,   4.08674921,
         5.39394066,   4.27848443,   4.43561393,   4.0765587 ,
         5.34416986,   5.78031265,   7.49311072,   8.99645058,
        20.94234447,  20.97204621,  13.65991386,  27.07540112,
        31.19693945,  21.61884596,  24.24336166,  35.07471669,
        58.43076189,  59.26530748,  71.03949712,  73.54794983,
        83.41214887,  88.31291879,  87.22727464,  97.07378643,
         1.49138706,   0.89482249,   1.28746238,   1.58982448,
         1.66089448,   1.54122167,   2.44849677,   2.75193459,
         2.98776545,   3.86669468,   5.24462502,   5.75620335,
         7.92161512,   8.82068841,  12.08428582,  13.04055777,
        16.40576504,  18.5306248 ,  25.04897239,  29.83396246,
        41.93671994,  38.58059139,  43.3366683 ,  52.07130935,
        50.03561919,  51.83069895,  52.20998053,  62.28504076,
         2.1257323 ,   2.05081502,   2.70101026,   2.992029  ,
         3.81410768,   5.34366563,   4.51630845,   5.00

In [118]:
i=0
all_target_list = sorted(glob.glob('drive/MyDrive/dacon/data/test_target/*.csv'))
for test_path in all_target_list:
    submit_df = pd.read_csv(test_path)
    submit_df['predicted_weight_g'] = pc[i*28:i*28+28]
    submit_df.to_csv(test_path, index=False)
    i+=1

In [113]:
testX = test.drop(['predicted_weight_g'],axis=1)
pc=[0]*140
for fold in range(27):
    pc += model[fold].predict(testX)/27
pc

array([  3.01170258,   2.43447941,   3.01695716,   3.06447166,
         5.24124034,   4.24585874,   4.82073063,   5.07003647,
         6.34455216,   7.13752782,   9.21185645,  11.09222984,
        21.85976362,  18.34478748,  15.17415464,  26.11417168,
        29.95707262,  18.89224654,  21.86781651,  34.50014341,
        56.16183102,  59.51034153,  64.44378638,  73.82482409,
        69.64607334,  79.00829458,  89.07148838,  91.62330961,
         1.96739211,   1.99003731,   2.06066132,   1.96649171,
         2.24228339,   2.58886527,   3.33507559,   3.54009748,
         4.59861435,   5.4739368 ,   6.72178707,   8.19720843,
         9.93016544,  10.97177356,  14.91226313,  16.14576828,
        17.33803368,  20.35244262,  23.87968314,  28.63453192,
        40.8439678 ,  42.04367971,  48.22319484,  56.93916976,
        65.3172698 ,  66.89253521,  67.89975262,  69.78226256,
         2.03766303,   2.00380371,   2.10139668,   2.60983974,
         2.72274116,   4.13835836,   3.33933745,   3.60

In [136]:
corr_df = train.corr()
corr_df = corr_df.apply(lambda x: round(x ,2))
s = corr_df.unstack()
성장 = pd.DataFrame(s['predicted_weight_g']).sort_values(by=0).fillna(0)
성장[0] = abs(성장[0])
성장.sort_values(by=[0],ascending=False).reset_index().to_csv('target값이랑corr.csv',index=False,encoding='utf-8-sig')
under_1per = 성장[성장[0]<0.1].index
under_1per

Index(['시간당백색광량20+21', '시간당백색광량19+20', '내부온도관측치20+21', '시간당총광량6+7',
       '내부온도관측치6+7', '내부온도관측치21+22', '내부온도관측치3+4', '시간당백색광량21+22',
       '시간당백색광량22+23', '내부온도관측치22+23', '시간당총광량18+19', '시간당백색광량0+1',
       '내부온도관측치5+6', '시간당청색광량12+13', '시간당적색광량13+14', '시간당적색광량14+15',
       '시간당청색광량11+12', '시간당적색광량17+18', '시간당적색광량16+17', '내부온도관측치4+5',
       '시간당적색광량15+16', '시간당적색광량18+19', '시간당적색광량0+1', '시간당적색광량19+20',
       '시간당청색광량13+14', '시간당총광량19+20', '시간당백색광량1+2', '시간당백색광량2+3', '시간당백색광량3+4',
       '시간당백색광량4+5'],
      dtype='object')

In [137]:
drop_column = train.copy()
drop_column = drop_column.drop(under_1per,axis=1)

In [138]:
drop_columntt = test.copy()
drop_columntt = drop_columntt.drop(under_1per,axis=1)

In [139]:
import catboost
X = drop_column.drop(['predicted_weight_g'],axis=1)
y = drop_column['predicted_weight_g']
model= {}
for f in range(0,27):
      train_idx = [i for i in range(0,784)]
      valid_idx = train_idx[f*28:f*28+28]
      for idx in valid_idx:
        train_idx.remove(idx)
      print(f'===================================={f+1}============================================')
      print(min(valid_idx),max(valid_idx))
      x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[valid_idx], y.iloc[train_idx], y.iloc[valid_idx]
      
      xgb_model = catboost.CatBoostRegressor(task_type='GPU',verbose=0,random_state=404)
      xgb_model.fit(x_train, y_train)
      
      y_pred = xgb_model.predict(x_val)
      mae = mean_squared_error(y_val, y_pred)**0.5
      print(f"{f + 1} Fold MAE = {mae}")
      model[f] = xgb_model
      print(f'================================================================================\n\n')

====================================1============================================
0 27
1 Fold MAE = 13.179910201796602


====================================2============================================
28 55
2 Fold MAE = 9.000843751136042


====================================3============================================
56 83
3 Fold MAE = 28.484937138368366


====================================4============================================
84 111
4 Fold MAE = 25.47397909975479


====================================5============================================
112 139
5 Fold MAE = 29.809235264575577


====================================6============================================
140 167
6 Fold MAE = 11.862091684427542


====================================7============================================
168 195
7 Fold MAE = 10.217299231051786


====================================8============================================
196 223
8 Fold MAE = 9.83803811763819


====================

In [140]:
testX = drop_columntt.drop(['predicted_weight_g'],axis=1)
pc=[0]*140
for fold in range(27):
    pc += model[fold].predict(testX)/27
pc

array([  3.18485283,   3.28137862,   3.33093165,   4.00247577,
         5.66998595,   4.49973597,   4.57834545,   4.20544039,
         5.29330057,   5.66296674,   7.32173883,   8.82698587,
        19.24597451,  22.15936324,  13.95713406,  26.12412482,
        32.72611656,  21.29429974,  25.76929139,  38.24442248,
        59.01789222,  59.74099335,  71.14557384,  76.41459777,
        84.3767177 ,  88.35201069,  87.5142546 ,  99.65182858,
         1.29370231,   0.64092929,   1.11298539,   1.33187778,
         1.48830625,   1.43300651,   2.47439033,   2.67834237,
         2.90387627,   3.79074222,   4.98894358,   5.4974565 ,
         7.23260073,   8.41232077,  11.68243834,  12.35003637,
        15.13870858,  17.5957035 ,  23.92737615,  28.05331839,
        41.34053609,  36.92382362,  41.39827768,  51.0425873 ,
        47.15067305,  48.98048069,  50.05367565,  61.61836077,
         1.7986024 ,   2.42376529,   2.59094409,   2.87803886,
         3.59454946,   4.94081311,   4.1582527 ,   4.33

In [143]:
i=0
all_target_list = sorted(glob.glob('drive/MyDrive/dacon/data/test_target/*.csv'))
for test_path in all_target_list:
    submit_df = pd.read_csv(test_path)
    submit_df['predicted_weight_g'] = pc[i*28:i*28+28]
    submit_df.to_csv(test_path, index=False)
    i+=1